In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

In [4]:
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw



In [5]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [8]:
class ConvNet(nn.Module):
  def __init__(self, num_classes=10):
    super(ConvNet, self).__init__()
    self.layer1 = nn.Sequential(
        nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
        nn.BatchNorm2d(16),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.layer2 = nn.Sequential(
        nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.fc = nn.Linear(7*7*32, num_classes)

  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = out.reshape(out.size(0), -1)
    out = self.fc(out)
    return out

In [9]:
model = ConvNet(num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

total_step = len(train_loader)
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)

    outputs = model(images)
    loss = criterion(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
            


Epoch [1/5], Step [100/600], Loss: 0.1674
Epoch [1/5], Step [200/600], Loss: 0.0513
Epoch [1/5], Step [300/600], Loss: 0.0377
Epoch [1/5], Step [400/600], Loss: 0.1019
Epoch [1/5], Step [500/600], Loss: 0.0284
Epoch [1/5], Step [600/600], Loss: 0.0422
Epoch [2/5], Step [100/600], Loss: 0.0345
Epoch [2/5], Step [200/600], Loss: 0.0603
Epoch [2/5], Step [300/600], Loss: 0.0460
Epoch [2/5], Step [400/600], Loss: 0.0330
Epoch [2/5], Step [500/600], Loss: 0.0114
Epoch [2/5], Step [600/600], Loss: 0.1126
Epoch [3/5], Step [100/600], Loss: 0.0576
Epoch [3/5], Step [200/600], Loss: 0.0620
Epoch [3/5], Step [300/600], Loss: 0.0436
Epoch [3/5], Step [400/600], Loss: 0.0198
Epoch [3/5], Step [500/600], Loss: 0.0086
Epoch [3/5], Step [600/600], Loss: 0.0554
Epoch [4/5], Step [100/600], Loss: 0.0225
Epoch [4/5], Step [200/600], Loss: 0.0145
Epoch [4/5], Step [300/600], Loss: 0.0177
Epoch [4/5], Step [400/600], Loss: 0.0129
Epoch [4/5], Step [500/600], Loss: 0.0118
Epoch [4/5], Step [600/600], Loss:

In [10]:
model.eval()
with torch.no_grad():
  correct = 0
  total = 0
  for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))



Test Accuracy of the model on the 10000 test images: 98.98 %


In [11]:
torch.save(model.state_dict(), 'model.ckpt')